In [1]:
import pandas as pd
from openslide import OpenSlide
from PIL import Image
import numpy as np
import os
from time import time
from natsort import natsorted

In [2]:
def svs2tiff(svs,rsf):
    src,fn = os.path.split(svs)
    fn,ext = os.path.splitext(fn)
    fn1 = fn + '.tif'
    if os.path.exists(os.path.join(wsi_dst,fn1)): return
    print(fn)
    svs_obj = OpenSlide(svs)
    svs_img = svs_obj.read_region(location=(0,0),level=0,size=svs_obj.level_dimensions[0]).convert('RGB')
    resize_factorx = rsf/float(svs_obj.properties['openslide.mpp-x']) #8um = 1.25x #4um = 2.5x, #2um=5x, 1um=10x, 0.5um=20x, 0.25um=40x
    resize_factory = rsf/float(svs_obj.properties['openslide.mpp-y'])
    resize_dimension = tuple([int(np.ceil(svs_obj.dimensions[0]/resize_factorx)),int(np.ceil(svs_obj.dimensions[1]/resize_factory))])
    svs_img = svs_img.resize(resize_dimension,resample=Image.NEAREST)
    svs_img.save(os.path.join(wsi_dst,fn1),resolution=1,resolution_unit=1,quality=100,compression=None)

In [3]:
lut = pd.read_excel(r"\\fatherserverdw\kyuex\imlist_all.xlsx")
healthylut = lut[lut['student score']>1]
healthylut

,index,gender,age,pid,path,filename,year,sample id,label name,sub label,...,Block or Slide?,"hypodermis (1=yes, 0=no)","healthy score (0=undecided, 1=cannot use, 2=some parts can be used, 3=one of the section can be used as whole, 4=more than one section, 5=all sections are free of damage)",score note,student score,ROI ANNOTATED (0 :not annotated 1.annotated),roi reviewed,Nroi,Inbal Score,Inbal comment
4,14,M,63.301370,8,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-07 17.10.29.ndpi,2,17359,02-17359,2,...,Both,1,2.0,collagen damage,2,1,1.0,NaN,NaN,NaN
13,15,M,63.301370,8,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-07 17.15.33.ndpi,2,17359,02-17359,1,...,Both,0,4.0,collagen damage?,4,1,1.0,NaN,4.0,left two are good within purple
14,53,M,64.147945,27,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-07 16.05.45.ndpi,3,4359,03-4359,3,...,Both,1,2.0,inflammation?,2,1,1.0,NaN,2.0,could use the left aspect of the left piece (b...
52,54,M,64.147945,27,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-09 20.12.48.ndpi,3,4359,03-4359,1,...,Both,1,NaN,NaN,2,1,1.0,NaN,NaN,NaN
53,55,M,64.147945,27,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-09 20.18.08.ndpi,3,4359,03-4359,2,...,Both,1,NaN,NaN,2,1,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156,1156,F,79.035616,476,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-07 14.56.13.ndpi,10,5202,10-5202,A2,...,Slide,1,2.0,physical damage to dermis,2,1,1.0,NaN,NaN,NaN
1157,1158,F,79.035616,476,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-08 16.34.23.ndpi,10,5202,10-5202,A3,...,Slide,1,2.0,collagen damage,2,1,1.0,NaN,NaN,NaN
1171,1172,M,68.202740,483,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-08 16.05.29.ndpi,10,6524,10-6524,A,...,Slide,1,2.0,inflamation,2,1,1.0,NaN,NaN,NaN
1172,1173,M,92.435616,484,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-07 17.24.15.ndpi,10,6634,10-6634,7,...,Slide,0,2.0,sun damage,2,1,1.0,NaN,NaN,NaN


In [4]:
wsi_src = r'\\fatherserverdw\kyuex\clue images'
wsi_dst = r'\\fatherserverdw\kyuex\clue images\1um'
wsis = [os.path.join(wsi_src,_) for _ in healthylut.filename]
wsis = natsorted(wsis)
len(wsis)

216

In [5]:
dstlist = [os.path.splitext(_)[0] for _ in os.listdir(r"\\fatherserverdw\kyuex\clue images\1um") if _.endswith('tif')]
srclist = [os.path.splitext(_)[0] for _ in healthylut.filename]
s = set(dstlist)
temp3 = [x for x in srclist if x not in s]
len(temp3)

162

In [6]:
wsis = [os.path.join(wsi_src,_+'.ndpi') for _ in temp3]
wsis = natsorted(wsis)
len(wsis)

162

In [7]:
wsis=wsis[50:100]
len(wsis)

50

In [ ]:
st = time()
for idx,wsi in enumerate(wsis):
    svs2tiff(wsi,1)
    print(idx,'/',len(wsis))
print("{:.2f} sec elapsed for {:d} images at 10x".format(time()-st,len(wsis)))

2022-06-09 22.00.47


C:\Users\labuser\AppData\Local\Temp\ipykernel_16532\688211719.py:12: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  svs_img = svs_img.resize(resize_dimension,resample=Image.NEAREST)


0 / 50
2022-06-09 22.12.56
1 / 50
2022-06-10 15.45.48
2 / 50
2022-06-10 15.47.34
